# Predicting heart disease using ML

This notebook looks into various Python-based ML and DS libraries in a attmpt
to buil a ML model capable of predicting wheather or not someone has heart disease based on their medical attributes.

Approach:
1. Problem definition;
2. Data;
3. Evaluation;
4. Features;
5. Modeling;
6. Experimentation.

## 1. Problem definition

### Given clinical paramaters about a patient, can the model predict wheather or not they have a heart disease?

## 2. Data

The original data came from the Cleavland data from the UCI Machine Learning Repository. https://archive.ics.uci.edu/ml/datasets/heart+Disease

There is also a version of it available on Kaggle. https://www.kaggle.com/ronitf/heart-disease-uci

## 3. Evaluation

Goal: If I can reach 95% accuracy at predicting whether or not a patient has 
heart disease during the proof of concept, we'll pursue the project.